In [ ]:
import cv2 as cv
import os
import matplotlib.pyplot as plt
import json
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('../')
from utils import *

annot = False
cls = ['good', 'tear']
folders = ["../train_test_splited_data/train", "../train_test_splited_data/test"]
# folders = ["train_test_splited_data/test"]

train_indexing = os.listdir(f"{folders[0]}/images")
print(train_indexing)
train_good_indexing = [i for i in train_indexing if 'good' in i]
print(len(train_good_indexing))
train_tear_indexing = [i for i in train_indexing if 'tear' in i]
print(len(train_tear_indexing))
train_good_indexing = [int(i.split('_')[-1].split('.')[0]) for i in train_good_indexing]
train_tear_indexing = [int(i.split('_')[-1].split('.')[0]) for i in train_tear_indexing]

test_indexing = os.listdir(f"{folders[1]}/images")
print(test_indexing)
test_good_indexing = [i for i in test_indexing if 'good' in i]
print(len(test_good_indexing))
test_tear_indexing = [i for i in test_indexing if 'tear' in i]
print(len(test_tear_indexing))
test_good_indexing = [int(i.split('_')[-1].split('.')[0]) for i in test_good_indexing]
test_tear_indexing = [int(i.split('_')[-1].split('.')[0]) for i in test_tear_indexing]



In [ ]:
if annot:
    for folder in folders:
        !rm -rf {folder}/annotations

    for folder in folders:
        if folder == folders[0]:
            good_indexing = train_good_indexing
            tear_indexing = train_tear_indexing
        else:
            good_indexing = test_good_indexing
            tear_indexing = test_tear_indexing
        Anotate(folder, good_indexing, cls[0])
        Anotate(folder, tear_indexing, cls[1])
else:
    pass

In [ ]:
for folder in folders:    
    index = 1
    tear_img_path = f"{folder}/images/tear_{index}.png"
    tear_txt_path1 = f"{folder}/labels/tear_{index}.txt"
    tear_txt_path2 = f"{folder}/predict/tear_{index}.txt"
    tear_json_path = f"{folder}/annotations/tear_{index}.json"

    txt_paths = []
    if os.path.exists(tear_txt_path1):
        txt_paths.append(tear_txt_path1)
    if os.path.exists(tear_txt_path2):
        txt_paths.append(tear_txt_path2)

    if txt_paths:
        draw_boxes_and_save_to_json(folder, tear_img_path, txt_paths, tear_json_path, index, cls[1], show=True, save=False)



In [ ]:
for folder in folders:
    print(len(os.listdir(f'{folder}/annotations/{cls[0]}/img')),len(os.listdir(f'{folder}/annotations/{cls[1]}/img')))

In [ ]:
def get_min_max_IoU(folder, cls):
    import re
    index = [int(re.findall(r'\d+', i)[0]) for i in os.listdir(f"{folder}/annotations/{cls}/anot")]
    min_IoU = 1
    max_IoU = 0
    for i in index:
        with open(f"{folder}/annotations/{cls}/anot/{i}.json") as json_file:
            data = json.load(json_file)
            for key in data:
                if data[key]['IoU'] < min_IoU:
                    min_IoU = data[key]['IoU']
                if data[key]['IoU'] > max_IoU:
                    max_IoU = data[key]['IoU']
    return min_IoU, max_IoU

for folder in folders:
    print(f"Folder: {folder}")
    min_IoU, max_IoU = get_min_max_IoU(folder, cls[0])
    # cls[0] = good
    print(min_IoU, max_IoU)
    min_IoU, max_IoU = get_min_max_IoU(folder, cls[1])
    # cls[1] = tear
    print(min_IoU, max_IoU)

In [ ]:
good_train_images = list_images(folders[0], cls[0], 0.75)
tear_train_images = list_images(folders[0], cls[1], 0.75)
good_test_images = list_images(folders[1], cls[0], 0.75)
tear_test_images = list_images(folders[1], cls[1], 0.75)